In [1]:
import os.path
from imutils import paths, resize
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

from custom import get_letters_list2, normalize_image_size

import pickle

import pandas as pd
from random import choices
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
data = []
labels = []
count = 0
for letter_path in paths.list_images('grainy_samples/letters'):
    image = cv2.imread(letter_path,0)
    
    # Resize image
    image = normalize_image_size(image,20,20)

    
    image = np.expand_dims(image, axis = 2)
    data.append(image)
    labels.append(letter_path.split(os.path.sep)[-2])
    count+=1
    if count%2000==0:
        print('Added image #%d'%count)


Added image #2000
Added image #4000
Added image #6000
Added image #8000
Added image #10000
Added image #12000
Added image #14000
Added image #16000
Added image #18000
Added image #20000
Added image #22000
Added image #24000
Added image #26000
Added image #28000
Added image #30000
Added image #32000
Added image #34000
Added image #36000
Added image #38000
Added image #40000
Added image #42000
Added image #44000
Added image #46000
Added image #48000
Added image #50000
Added image #52000
Added image #54000
Added image #56000
Added image #58000
Added image #60000
Added image #62000
Added image #64000
Added image #66000
Added image #68000
Added image #70000
Added image #72000
Added image #74000
Added image #76000
Added image #78000
Added image #80000
Added image #82000
Added image #84000
Added image #86000
Added image #88000
Added image #90000
Added image #92000
Added image #94000


In [3]:
data = np.array(data,dtype='float')/255.0
labels = np.array(labels)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(data,labels, test_size=0.25)

In [5]:
lb = LabelBinarizer().fit(y_train)
y_train = lb.transform(y_train)
y_test = lb.transform(y_test)

In [7]:
model = Sequential()
model.add(Conv2D(10, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Conv2D(40, (5, 5), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dense(19, activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=19, epochs=2, verbose=1)

Train on 71238 samples, validate on 23746 samples
Epoch 1/2
71238/71238 [==============================] - 102s 1ms/step - loss: 0.1771 - acc: 0.9529 - val_loss: 0.0593 - val_acc: 0.9846
Epoch 2/2
71238/71238 [==============================] - 115s 2ms/step - loss: 0.0481 - acc: 0.9870 - val_loss: 0.0553 - val_acc: 0.9864


In [8]:
model.save('models/ideal/ideal.h5')

with open('models/ideal/model_label_map.dat','wb') as f:
    pickle.dump(lb,f)